In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('mode.chained_assignment', None)

df = pd.read_csv(r"E:\Tung\Python\VcreamProject\cache\ALL.csv")

df.drop(['Unnamed: 0'], axis = 1, inplace=True)

In [33]:
df

,Ticker,Date,Open,High,Low,Close,Volume
0,A32,2018-11-29,22.9056,22.9056,22.9056,22.9076,1000
1,A32,2018-12-03,22.9056,22.9056,22.9056,22.9076,100
2,A32,2018-12-04,20.0424,20.0424,20.0424,20.0441,2000
3,A32,2018-12-21,21.6172,21.6172,21.6172,21.6190,100
4,A32,2019-01-02,22.9056,22.9056,22.9056,22.9076,100
...,...,...,...,...,...,...,...
2276360,YTC,2022-05-12,68.0000,68.0000,68.0000,68.0000,100
2276361,YTC,2022-06-15,66.0000,66.0000,65.0000,65.0000,200
2276362,YTC,2022-06-24,65.5000,65.5000,65.5000,65.5000,10
2276363,YTC,2022-09-05,65.5000,65.5000,65.5000,65.5000,200


In [38]:
vcb = df.loc[df['Ticker'] == 'VCB']
vcb 

,Ticker,Date,Open,High,Low,Close,Volume
2028904,VCB,2009-06-30,16.2420,16.2420,16.2420,16.2452,294070
2028905,VCB,2009-07-01,17.0541,17.0541,16.1067,16.3806,6248390
2028906,VCB,2009-07-02,16.1067,16.2420,15.5653,15.7037,1515670
2028907,VCB,2009-07-03,15.2946,15.4299,15.1592,15.1622,899720
2028908,VCB,2009-07-06,15.1592,15.8360,15.1592,15.8391,1571740
...,...,...,...,...,...,...,...
2032221,VCB,2022-10-12,63.0000,65.3000,62.2000,64.0000,1517400
2032222,VCB,2022-10-13,64.1000,66.5000,62.7000,66.5000,1596800
2032223,VCB,2022-10-14,68.5000,68.5000,67.3000,68.1000,1599300
2032224,VCB,2022-10-17,67.3000,68.0000,65.5000,66.3000,1241400


In [56]:
vcb['previous_high'] = vcb['High'].shift(1)
vcb['previous_low'] = vcb['Low'].shift(1)
vcb['previous_close'] = vcb['Close'].shift(1)

vcb['high-low'] = vcb['High'] - vcb['Low']
vcb['high-pc'] = np.abs(vcb['High'] - vcb['previous_close'])
vcb['low-pc'] = np.abs(vcb['Low'] - vcb['previous_close'])
vcb['true_range'] = np.max(vcb[['high-low', 'high-pc', 'low-pc']], axis=1)
vcb['true_range_10'] = vcb['true_range'].rolling(10).mean()

vcb['+dm'] = np.where(((vcb['High'] - vcb['previous_high']) > (vcb['previous_low'] - vcb['Low'])) & (vcb['High'] - vcb['previous_high'] > 0), vcb['High'] - vcb['previous_high'], 0)
vcb['-dm'] = np.where(((vcb['High'] - vcb['previous_high']) < (vcb['previous_low'] - vcb['Low'])) & (vcb['previous_low'] - vcb['Low'] > 0), vcb['previous_low'] - vcb['Low'], 0)

vcb['+di'] = 100 * vcb['+dm'].ewm(alpha=1/10).mean()/vcb['true_range_10']
vcb['-di'] = 100 * vcb['-dm'].ewm(alpha=1/10).mean()/vcb['true_range_10']

vcb['dx'] = np.abs((vcb['+di'] - vcb['-di'])/(vcb['+di'] + vcb['-di']))*100
vcb['adx_10'] = vcb['dx'].ewm(min_periods=0, com=10, adjust=False).mean()
vcb

,Ticker,Date,Open,High,Low,Close,Volume,previous_high,previous_low,previous_close,...,high-pc,low-pc,true_range,true_range_10,+dm,-dm,+di,-di,dx,adx_10
2028904,VCB,2009-06-30,16.2420,16.2420,16.2420,16.2452,294070,NaN,NaN,NaN,...,NaN,NaN,0.0000,NaN,0.0000,0.0000,NaN,NaN,NaN,NaN
2028905,VCB,2009-07-01,17.0541,17.0541,16.1067,16.3806,6248390,16.2420,16.2420,16.2452,...,0.8089,0.1385,0.9474,NaN,0.8121,0.0000,NaN,NaN,NaN,NaN
2028906,VCB,2009-07-02,16.1067,16.2420,15.5653,15.7037,1515670,17.0541,16.1067,16.3806,...,0.1386,0.8153,0.8153,NaN,0.0000,0.5414,NaN,NaN,NaN,NaN
2028907,VCB,2009-07-03,15.2946,15.4299,15.1592,15.1622,899720,16.2420,15.5653,15.7037,...,0.2738,0.5445,0.5445,NaN,0.0000,0.4061,NaN,NaN,NaN,NaN
2028908,VCB,2009-07-06,15.1592,15.8360,15.1592,15.8391,1571740,15.4299,15.1592,15.1622,...,0.6738,0.0030,0.6768,NaN,0.4061,0.0000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2032221,VCB,2022-10-12,63.0000,65.3000,62.2000,64.0000,1517400,66.0000,62.0000,62.0000,...,3.3000,0.2000,3.3000,2.85,0.0000,0.0000,3.419591,39.469053,84.053630,57.525406
2032222,VCB,2022-10-13,64.1000,66.5000,62.7000,66.5000,1596800,65.3000,62.2000,64.0000,...,2.5000,1.3000,3.8000,3.03,1.2000,0.0000,6.855198,33.411921,65.951385,58.291404
2032223,VCB,2022-10-14,68.5000,68.5000,67.3000,68.1000,1599300,66.5000,62.7000,66.5000,...,2.0000,0.8000,2.0000,3.04,2.0000,0.0000,12.728331,29.971812,40.382725,56.663342
2032224,VCB,2022-10-17,67.3000,68.0000,65.5000,66.3000,1241400,68.5000,67.3000,68.1000,...,0.1000,2.6000,2.6000,2.95,0.0000,1.8000,11.804987,33.899281,48.341860,55.906844


In [55]:
vcb['dx'].ewm(min_periods=0, com=10, adjust=False).mean().head()

2028904   NaN
2028905   NaN
2028906   NaN
2028907   NaN
2028908   NaN
Name: dx, dtype: float64